In [1]:
import numpy as np
import pandas as pd

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf

## Helper Functions (board to one hot representation)

In [2]:
import pandas as pd
import numpy as np

def is_correct(p):
    """Are all of the rows, cols, and boxes completed (1-9 entries)"""
    full = set(range(1,10))
    for c in range(0,9):
        for r in range(0,9): 
            for get_func in get_rect:
                if len(set(get_func(p,r,c)).intersection(full)) != 9:
                    return False
    return True



def is_complete(p):
    """Are all of the rows, cols, and boxes non zero"""
    for c in range(0,9):
        for r in range(0,9): 
            if p[r,c] == 0:
                return False                
    return True

def create_matrix():
    return np.zeros(dtype=int, shape=(9,9))

def to_matrix(p):
    return np.array([np.array([int(i) for i in r]) for r in p.split('\n')])

# helper functions to pull associated rows, columns and boxes for a given cell.
get_row = lambda p,x,y: (p[x]).flatten()
get_col = lambda p,x,y: (p[:,y]).flatten()
get_box = lambda p,x,y: (p[3*(x/3):3*(x/3)+3,3*(y/3):3*(y/3)+3]).flatten()

get_rect = [get_row, get_col, get_box]

def enumerate_rows(p):
    result = []
    for i in range(0, 9):
        result.append(p[i].flatten())
    return result

def enumerate_columns(p):
    result = []
    for i in range(0, 9):
        result.append(p[:,i].flatten())
    return result

def enumerate_boxes(p):
    result = []
    for x in range(0, 9, 3):
        for y in range(0, 9, 3):
            result.append(get_box(p,x,y))
    return result

def get_possibilities(p):
    """For each entry in the grid, list out all of the things it could be"""
    full = set(range(1,10))
    possibiliteis = np.array([[set() for x in range(1, 10)] for x in range(1, 10)])
    for c in range(0,9):
        for r in range(0,9):
            candidates = set(full)
            if p[r,c] == 0:   
                for get_func in get_rect:
                    [candidates.remove(x) for x in set(get_func(p,r,c)) if x in candidates]
                possibiliteis[r,c] = candidates
            else:
                possibiliteis[r,c] = set([p[r,c]])
    return possibiliteis

def solve_step(p):
    """Attempt to solve the sudoku puzzle using 2 stratigies"""
    
    solution = create_matrix()

    is_update = False
    possibiliteis = get_possibilities(p)

    # if there is only one possible thing a grid item could be, then we have our answer!
    for c in range(0,9):
        for r in range(0,9):
            if p[r,c] == 0:   
                if len(possibiliteis[r,c]) == 1:
                    solution[r,c] = list(possibiliteis[r,c])[0]
                    is_update = True

    # This is sort of the opposite of the above
    # If this is the only cell in the row, col, box than can be a number - then make it so
    for c in range(0,9):
        for r in range(0,9):
            if p[r,c] == 0:    
                for get_func in get_rect:
                    #for each possibility, see if this is the only cell that can satisfy it
                    func_poss = [s for s in get_func(possibiliteis,r,c)]
                    func_poss.remove(possibiliteis[r,c])
                    poss = set(range(1,10))

                    for s in func_poss:
                        for e in s:
                            if e in poss:
                                poss.remove(e)
                    if len(poss) == 1:
                        solution[r,c] = poss.pop()
                        is_update = True
    if is_update:
        return solution
    else:
        return None

class Count(object):
    def __init__(self):
        self.c=0
    def increment(self):
        self.c += 1
    def get(self):
        return self.c

def solve(px, solution_list=None, step=0, count=None):
    if count is None:
        count = Count()
    count.increment()
    
    if step > 100 or count.get() > 1000:
        # sometimes we get unlucky and istead of waiting for
        # the very hard puzzle to complete we shortcircuit 
        raise ValueError('oops - unlucky branch in solver')

            
    p = px.copy()
    
    solution = solve_step(p)

    if solution is not None:
        if solution_list is None:
            solution_list = []

    
    while solution is not None:
        p += solution
        solution_list.append(p)
        solution = solve_step(p)
        
    if is_complete(p):
        if is_correct(p):
            return p, solution_list
        else:
            return None, None
    
    # find the first cell with multiple possibilities
    # make a guess!
    possibiliteis = get_possibilities(p)

    r_range = np.arange(9)
    c_range = np.arange(9)

    np.random.shuffle(r_range)
    np.random.shuffle(c_range)

    is_done = True
    for row in r_range:
        for col in c_range:
            poss_list = np.array(list(possibiliteis[row, col]))
            if len(poss_list) > 1:
                is_done = True
                break
        if is_done:
            break

    np.random.shuffle(poss_list)

    if len(poss_list) <= 1:
        return None, None

    for poss in poss_list:
        p[row, col] = poss

        if solution_list is not None:
            rs, slns = solve(p.copy(), solution_list + [p.copy()], step+1, count)
        else:
            rs, slns = solve(p.copy(), solution_list, step+1, count)

        if rs is not None:
            if is_complete(rs):
                if is_correct(rs):
                    return rs, slns
        p[row, col] = 0
                               
    return None, None



In [3]:
def matrix_to_one_hot(m):
    '''Make a 1-hot encoded version of an input sudoku matrix'''
    ret = np.zeros(9*9*9, dtype=int)
    for i, v in enumerate(m.flatten()):
        if v == 0:
            continue
        ret[i*9+v-1] = 1 
    return ret

def set_matrix_to_one_hot(m):
    '''Make a 1-hot encoded version of an input sudoku matrix'''
    ret = np.zeros(9*9*9, dtype=int)
    for i, s in enumerate(m.flatten()):
        for v in s:
            if v == 0:
                continue
            ret[i*9+v-1] = 1 
    return ret

def one_hot_to_set_matrix(oh):
    '''Make convert a one hot encoded sudoku matrix into a sudoku 
    matrix'''
    ret = []
    
    for i in range(81):
        val = -1
        ret.append(list())
        for v in range(9):
            if oh[i*9+v] > 0:
                val = oh[i*9+v]
            
                ret[i].append(v + 1)
                
    return ret

def one_hot_to_matrix(oh):
    '''Make convert a one hot encoded sudoku matrix into a sudoku 
    matrix'''
    ret = np.zeros(81, dtype=int)
    for i in range(81):
        val = -1
        for v in range(9):
            if oh[i*9+v] != 0:
                val = oh[i*9+v]
            if val != -1:
                ret[i] = v + 1
                break
    return ret.reshape(9,9)

def vec_to_one_hot(m):
    '''1 hot encode a vecotr of numbers between 0 and 9'''
    ret = np.zeros(9*9, dtype=int)
    for i, v in enumerate(m.flatten()):
        if v == 0:
            continue
        ret[i*9+v-1] = 1 
    return ret

## Build training data

In [4]:

from collections import defaultdict

def generate_batch(n):
    '''Generate a set of sudoku puzzle steps with the next possible 
    moves
    
    Returns:
        puz_x_features: the one hot encodeing of each 'box'
            by 'box' i mean a 1 hot encoding of each
                row (num 1-9)
                column (num 1-9)
                9x9 box (num 1-9)
        puz_y: the one hot encoded version of the next possible moves
        puz: the puzzle step (nice formatting)
    '''
    puzzles = []

    for _ in range(n):
        # create a sudoku puzzle, solve it, list each step
        #answer, steps to get to answer
        result, steps = sudoku.solve(sudoku.create_matrix())
        puzzle = steps
        
        # make sure the puzzle exists
        if puzzle is None:
            continue
        #sanity checks
        elif (not (is_complete(puzzle[-1]) and is_correct(puzzle[-1]))):
            continue
        else:
            puzzles.append(puzzle)
    
    puz = []
    puz_y = []
    
    f1 = lambda x: x
    f2 = lambda x: np.flip(x, 1)
    f3 = lambda x: np.flip(x, 0)
    
    flips = [f1, f2, f3]
    
    # rot will rotate the matrix, this gives us cheap (CPU) 
    # training data
    for rot in range(0,4):
        for flip in flips:
            for puzzle in puzzles:
                previous = flip(np.rot90(puzzle[0], rot))
                for step in puzzle[1:-2]:
                    step = flip(np.rot90(step, rot))
                    if (step > 0).sum() == (previous > 0).sum():
                        continue

                    
                    # the delta between current and previous steps
                    # are the potential answers for this step
                    
                    puz_y.append(step - previous)

                    puz.append(previous)
                    previous = step
                
    return puz, puz_y


In [5]:
#!rm sudoku.dat

In [6]:
!tail -n 1 sudoku.dat

2,6,4,7,9,5,3,8,1,1,0,3,2,8,6,0,0,0,0,0,9,4,1,3,0,0,0,6,3,5,9,2,1,0,0,0,9,4,8,3,6,7,0,1,0,7,2,1,8,5,4,9,3,6,4,0,0,6,0,0,1,0,0,0,0,6,1,7,0,0,0,0,3,1,0,5,0,8,0,0,0	0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,4,0,0,0,0


In [7]:
#!sort -R sudoku.dat > sudoku_rnd.dat

In [8]:
!ls -alh sudoku_rnd.dat

-rw-rw-r-- 1 trophi trophi 1.7G Nov 25 15:44 sudoku_rnd.dat


In [9]:
def generate_file_batch(n, f):
    retx = []
    rety = []
    for _ in range(n):
        xx, yy = f.readline().split('\t')
        retx.append(np.array([int(v) for v in xx.split(',')]).reshape((9,9)))
        rety.append(np.array([int(v) for v in yy.split(',')]).reshape((9,9)))
        
    return retx, rety
            
def get_x_y(f, batches=1):
    
    puz, puz_y = generate_file_batch(batches, f)
    x = list([matrix_to_one_hot(v) for v in puz])
    
    #y = [[y.sum() > 110, y.sum() <= 110] for y in puz]
    #if is_possible:
    #    y = list([set_matrix_to_one_hot(sudoku.get_possibilities(v)) for v in puz])
    #else:
    y = list([matrix_to_one_hot(v) for v in puz_y])
    
    return x, y

## Tensorflow Helpers

In [10]:
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.0, shape=shape)
    return tf.Variable(initial)

In [12]:
def nn_layer(input_tensor, input_dim, output_dim, layer_name, act, batch_norm):
    """Reusable code for making a simple neural net layer.

    It does a matrix multiply, bias add, and then uses relu to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read,
    and adds a number of summary ops.
    """
    
    epsilon = 1e-3
    
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
        # This Variable will hold the state of the weights for the layer
        with tf.name_scope('weights'):
            weights = weight_variable([input_dim, output_dim])

            variable_summaries(weights)
        with tf.name_scope('biases'):
            biases = bias_variable([output_dim])
            variable_summaries(biases)
        with tf.name_scope('Wx_plus_b'):
            preactivate = tf.matmul(input_tensor, weights) + biases
            tf.summary.histogram('pre_activations', preactivate)
            
        if batch_norm:
            with tf.name_scope('batch_norm'):
                batch_mean, batch_var = tf.nn.moments(preactivate,[0])
                scale = tf.Variable(tf.ones([output_dim]))
                beta = tf.Variable(tf.zeros([output_dim]))

                variable_summaries(scale)
                variable_summaries(beta)

                preactivate = tf.nn.batch_normalization(
                    preactivate, batch_mean, batch_var, beta, scale, epsilon)    


        activations = act(preactivate, name='activation')
            
        tf.summary.histogram('activations', activations)
    return activations

def network(input_variables, keep_prob, hidden_sizes, output_size, name, batch_norm):
    # concat all the shared weights
    past_input = tf.concat(values=input_variables, axis=0)
    
    
    ops= []
    past_size = sum(v.shape[1].value for v in input_variables)
    
    with tf.name_scope('network_{}'.format(name)):
        for i, size in enumerate(hidden_sizes):

            layer = nn_layer(past_input, 
                             past_size, 
                             size, 
                             'layer_{}'.format(i), 
                             act=tf.nn.tanh, 
                             batch_norm=batch_norm)
            if keep_prob is not None:
                with tf.name_scope('dropout'):
                    tf.summary.scalar('dropout_keep_probability', keep_prob)
                    dropped = tf.nn.dropout(layer, keep_prob)
                    past_input = dropped
            else:
                past_input = layer
                
            past_size = size
            

            
        y = nn_layer(past_input, past_size, output_size, 'layer_final', 
                     act=tf.identity, batch_norm=False)
    
    return y

In [16]:
with open('sudoku_rnd.dat', 'r') as f:
    puz_x, puz_y = get_x_y(f)


input_size = len(puz_x[0])
output_size = len(puz_y[0])
hidden_size = int(output_size+(2. / 3. * input_size) * 1.5) 

hidden_sizes = [hidden_size * 3, hidden_size * 3]

print input_size
print hidden_size
print output_size

#9 (per box) * 9 (per value) * 9 (boxes) * 3 (box types)
#print 9 * 9 * 9 * 3

# hidden input rule of thumb tradeoff between num examples and hidden units
# https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw
100000.*30/((9*9+sum(hidden_sizes))*5)

729
1458
729


67.95786612300374

In [17]:
tf.reset_default_graph()

x  = tf.placeholder(tf.float32, shape=[None, input_size])
y_ = tf.placeholder(tf.float32, shape=[None, output_size])

dropout_prob = tf.placeholder_with_default(1.0, shape=())

net = network([x], keep_prob=dropout_prob,
              hidden_sizes=hidden_sizes, 
              output_size=output_size, 
              name='grid_sum_network', batch_norm=True)

with tf.name_scope('cross_entropy'):
    
    cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(
        labels=y_, logits=net)

    loss = tf.reduce_mean(cross_entropy)
    tf.summary.scalar('cross_entropy', loss)


with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(
        learning_rate=0.5).minimize(loss)

In [19]:
root_dir = model_file = "/home/trophi/notebook/Notebooks/deep_sudoku"
summary_dir = root_dir + '/summary'

# gpu was OOMing
session_conf = tf.ConfigProto(
    device_count={'CPU' : 1, 'GPU' : 0},
    allow_soft_placement=True,
    log_device_placement=True
)

sess = tf.Session(config=session_conf)

!rm -rf '/home/trophi/notebook/Notebooks/deep_sudoku/summary/test'
!rm -rf '/home/trophi/notebook/Notebooks/deep_sudoku/summary/train'

# Merge all the summaries and write them out to /tmp/mnist_logs (by default)
merged = tf.summary.merge_all()
#train_writer = tf.summary.FileWriter(summary_dir + '/train')
test_writer = tf.summary.FileWriter(summary_dir + '/test')

test_writer.get_logdir()

'/home/trophi/notebook/Notebooks/deep_sudoku/summary/test'

In [20]:
init_op = tf.global_variables_initializer()

sess.run(init_op)

i = 0
f = open('sudoku_rnd.dat', 'r')

In [23]:
test_puz_x, test_puz_y = get_x_y(f, 700)

In [ ]:
num_steps = 2000000

for _ in range(num_steps):
    if f.closed:
        break

    puz_x, puz_y = get_x_y(f, 30)
    if i % 100 == 1:  
        summary = sess.run(
            merged, 
            feed_dict={x: test_puz_x, y_:test_puz_y, dropout_prob:1.0}
        )
        test_writer.add_summary(summary, i)
        #print('Accuracy at step %s: %s' % (i, acc))
    else:
        sess.run(train_step, feed_dict={x: puz_x, y_:puz_y, dropout_prob:0.5})
    i += 1   

In [ ]:
f.close()

In [29]:
with open('sudoku_rnd.dat', 'r') as l:
    puz_x, puz_y = get_x_y(l, 30)
    res = sess.run(net, feed_dict={x: puz_x})

In [30]:
r = (res > 0).astype(int)

In [44]:
res[0].shape

(729,)

In [45]:
def best_guess(s):
    res = np.zeros(shape=729)
    res[np.argmax(s)] = 1
    return res

In [46]:
one_hot_to_matrix(best_guess(res[0]))

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 6, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [58]:
k = 13

one_hot_to_matrix(puz_x[k])

array([[0, 4, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 3, 0, 0, 0, 0, 0, 0],
       [0, 9, 7, 2, 0, 0, 0, 8, 1],
       [2, 0, 0, 0, 0, 8, 0, 0, 7],
       [0, 0, 9, 0, 0, 0, 2, 0, 0],
       [0, 0, 6, 3, 0, 2, 0, 0, 9],
       [9, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 4, 0, 2, 0, 0, 0, 0],
       [5, 0, 2, 0, 0, 0, 0, 0, 3]])

In [130]:
solve(one_hot_to_matrix(puz_x[k]))[0]

array([[1, 4, 5, 8, 9, 6, 3, 7, 2],
       [8, 2, 3, 4, 1, 7, 9, 5, 6],
       [6, 9, 7, 2, 3, 5, 4, 8, 1],
       [2, 3, 1, 9, 5, 8, 6, 4, 7],
       [7, 8, 9, 6, 4, 1, 2, 3, 5],
       [4, 5, 6, 3, 7, 2, 8, 1, 9],
       [9, 7, 8, 1, 6, 3, 5, 2, 4],
       [3, 1, 4, 5, 2, 9, 7, 6, 8],
       [5, 6, 2, 7, 8, 4, 1, 9, 3]])

In [59]:
zip(one_hot_to_matrix(best_guess(res[k])), one_hot_to_matrix(puz_y[k]))

[(array([0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 5, 0, 6, 0, 0, 0, 0])),
 (array([0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0])),
 (array([0, 0, 0, 0, 0, 0, 0, 0, 0]), array([6, 0, 0, 0, 0, 0, 0, 0, 0])),
 (array([0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 1, 0, 0, 0, 0, 0, 0])),
 (array([0, 0, 0, 6, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0])),
 (array([0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0])),
 (array([0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 8, 0, 0, 0, 0, 0, 0])),
 (array([0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0])),
 (array([0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0]))]

In [128]:
solve(one_hot_to_matrix(puz_x[k]))[0]

array([[1, 4, 5, 9, 8, 7, 3, 6, 2],
       [8, 2, 3, 4, 1, 6, 9, 7, 5],
       [6, 9, 7, 2, 5, 3, 4, 8, 1],
       [2, 3, 1, 5, 9, 8, 6, 4, 7],
       [4, 5, 9, 6, 7, 1, 2, 3, 8],
       [7, 8, 6, 3, 4, 2, 5, 1, 9],
       [9, 6, 8, 1, 3, 5, 7, 2, 4],
       [3, 1, 4, 7, 2, 9, 8, 5, 6],
       [5, 7, 2, 8, 6, 4, 1, 9, 3]])

In [105]:
for i in range(100):
    try:
        puz_x, puz_y = get_x_y()
    except:
        continue
    l = sess.run(train_step, 
                    feed_dict={x: puz_x, y_: puz_y})
    if i % 10 == 0:
        print sess.run(accuracy, feed_dict={x: puz_x, y_: puz_y})

0.6125
0.75
0.858974
1.0
0.944444
0.869048
0.911111
0.841667
0.825


In [108]:
l

In [103]:
loss

In [92]:
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(net, 1), tf.argmax(y_, 1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar('accuracy', accuracy)

# legacy

In [148]:
!rm example.log

model_file = "/home/trophi/notebook/Notebooks/deep_sudoku/model/model.ckpt"

saver = tf.train.Saver()


sess.run(init_op)
num_slns = 0
# 300000 * batch(30 * 10 * 4)
for i in range(100):

    try:
        puz_x_features, puz_poss, puz_y, puz = generate_batch(1)
    except:
        logging.debug(str(time.localtime()))
        logging.debug("exception {}".format(i))
        continue
    #train_result = train_step.run(feed_dict={x: puz_x_features, y_: puz_y})
    num_slns += len(puz)
    loss = sess.run(train_step, 
                    feed_dict={x: puz_x_features, y_: puz_poss})
    if i % 100 == 0:
        save_path = saver.save(sess, model_file)
        logging.debug(str(time.localtime()))

        logging.debug("loss - {}".format(l))
        logging.debug("{} saved: {}".format(i, save_path))


rm: cannot remove 'example.log': No such file or directory


In [148]:
!rm example.log

model_file = "/home/trophi/notebook/Notebooks/deep_sudoku/model/model.ckpt"

import logging
import time
LOG_FILENAME = 'example.log'
logging.basicConfig(filename=LOG_FILENAME,level=logging.DEBUG)

saver = tf.train.Saver()
sess = tf.Session(config=session_conf)

sess.run(init_op)
num_slns = 0
# 300000 * batch(30 * 10 * 4)
for i in range(100):

    try:
        puz_x_features, puz_poss, puz_y, puz = generate_batch(1)
    except:
        logging.debug(str(time.localtime()))
        logging.debug("exception {}".format(i))
        continue
    #train_result = train_step.run(feed_dict={x: puz_x_features, y_: puz_y})
    num_slns += len(puz)
    loss = sess.run(train_step, 
                    feed_dict={x: puz_x_features, y_: puz_poss})
    if i % 100 == 0:
        save_path = saver.save(sess, model_file)
        logging.debug(str(time.localtime()))

        logging.debug("loss - {}".format(l))
        logging.debug("{} saved: {}".format(i, save_path))


rm: cannot remove 'example.log': No such file or directory


In [ ]:



input_weights = []

# the weights for 1 hot encoded input values can be shared as 
# the logic for box, row, column should all be the same
with tf.variable_scope("w") as scope:
    for i in range(9*3):
        
        w = tf.get_variable("weights", [9*9, h1_size],
                initializer=tf.truncated_normal_initializer())
        if i == 0:
            scope.reuse_variables() 
        input_weights.append(w)
        
# concat all the shared weights
w1 = tf.concat(values=input_weights, axis=0)

# normal NN after this point
# perhaps ill add another hidden layer at some point
b1 = tf.Variable(tf.truncated_normal([h1_size], dtype=tf.float32))
h1 = tf.nn.sigmoid(tf.matmul(x, w1) + b1)


w2 = tf.Variable(tf.truncated_normal([h1_size, h2_size], dtype=tf.float32))
b2 = tf.Variable(tf.truncated_normal([h2_size], dtype=tf.float32))
h2 = tf.nn.sigmoid(tf.matmul(h1, w2) + b2)

w3 = tf.Variable(tf.truncated_normal([h2_size, output_size], dtype=tf.float32))
b3 = tf.Variable(tf.truncated_normal([output_size], dtype=tf.float32))



y = tf.matmul(h2, w3) + b3

# this allows us to guess for multiple cells at once
# softmax would allow us only to pick one
# the training data is set up to calculate all possible moves at each step
# so this is important

cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_, logits=y)

#loss = tf.reduce_mean(cross_entropy)

# arbitrary choice, need to learn more about this
train_step = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cross_entropy)

# gpu was OOMing
session_conf = tf.ConfigProto(
    device_count={'CPU' : 1, 'GPU' : 0},
    allow_soft_placement=True,
    log_device_placement=True
)

init_op = tf.global_variables_initializer()

In [103]:
!rm example.log

model_file = "/home/trophi/notebook/Notebooks/deep_sudoku/model/model.ckpt"

import logging
import time
LOG_FILENAME = 'example.log'
logging.basicConfig(filename=LOG_FILENAME,level=logging.DEBUG)

saver = tf.train.Saver()
with tf.Session(config=session_conf) as sess:
    sess.run(init_op)
    num_slns = 0
    # 300000 * batch(30 * 10 * 4)
    for i in range(100):
        
        try:
            puz_x_features, puz_y, puz = generate_batch(1)
        except:
            logging.debug(str(time.localtime()))
            logging.debug("exception {}".format(i))
            continue
        #train_result = train_step.run(feed_dict={x: puz_x_features, y_: puz_y})
        num_slns += len(puz)
        loss = sess.run(train_step, 
                        feed_dict={x: puz_x_features, y_: puz_y})
        if i % 100 == 0:
            save_path = saver.save(sess, model_file)
            logging.debug(str(time.localtime()))

            logging.debug("loss - {}".format(l))
            logging.debug("{} saved: {}".format(i, save_path))


rm: cannot remove 'example.log': No such file or directory


In [111]:
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session(config=session_conf) as sess:
    # Restore variables from disk.
    saver.restore(sess, model_file)
    print("Model restored.")
    puz_x_features, puz_y, puz = generate_batch(1)
    res = sess.run(y, feed_dict={x: puz_x_features, y_: puz_y})
    
    

INFO:tensorflow:Restoring parameters from /home/trophi/notebook/Notebooks/deep_sudoku/model/model.ckpt
Model restored.


In [112]:
zip(res, puz_y)

[(array([ 166.10456848], dtype=float32), [69]),
 (array([ 167.82572937], dtype=float32), [73]),
 (array([ 157.35386658], dtype=float32), [77]),
 (array([ 158.49708557], dtype=float32), [86]),
 (array([ 158.96270752], dtype=float32), [89]),
 (array([ 162.67066956], dtype=float32), [95]),
 (array([ 167.88809204], dtype=float32), [101]),
 (array([ 164.58763123], dtype=float32), [105]),
 (array([ 166.92747498], dtype=float32), [111]),
 (array([ 170.36647034], dtype=float32), [118]),
 (array([ 160.38208008], dtype=float32), [119]),
 (array([ 156.81100464], dtype=float32), [121]),
 (array([ 159.48487854], dtype=float32), [125]),
 (array([ 149.81484985], dtype=float32), [126]),
 (array([ 145.41241455], dtype=float32), [144]),
 (array([ 147.80119324], dtype=float32), [151]),
 (array([ 159.54051208], dtype=float32), [158]),
 (array([ 150.80543518], dtype=float32), [205]),
 (array([ 154.25408936], dtype=float32), [209]),
 (array([ 151.75422668], dtype=float32), [229]),
 (array([ 143.99935913], d

In [56]:
non_zero = []
is_empty = []
for r, p in zip(np.argmax(res, axis=1), puz):
    one_hot = np.zeros(729)
    
    one_hot[r] = 1
    ans = one_hot_to_matrix(one_hot)
    
    non_zero.append((p == 0).sum())
    is_empty.append(((p == 0) & (ans != 0) ).sum())

print np.mean(non_zero)/81
print np.mean(is_empty)

0.568724279835
0.6


In [57]:
np.argmax(res, axis=1)

array([246,  19, 246, 246, 246, 180, 180, 180, 246, 180, 180, 246, 417,
       417, 180, 337, 337, 337, 337, 337, 337, 337, 488, 369, 113, 337,
       246, 417, 180, 246, 246, 246, 246, 246, 246, 180, 417, 180, 180,
       180, 180, 180, 369, 369, 369,  19, 337,  19, 337, 369, 180,  19,
        19, 456, 456, 456, 456, 635, 635, 246, 337, 337, 180, 337, 337,
       374, 337, 705, 337, 337, 705, 180, 180, 417, 417,   6,   6, 369,
       369, 369, 246, 551, 417, 373, 373, 523, 523, 373, 369, 373, 369,
       180, 705, 337, 488, 337, 337, 369, 337, 180, 705, 705, 369,  19,
       369, 246, 246, 246, 246, 246, 180, 417, 180, 180, 180, 180, 180,
       369, 369, 369, 337, 337, 337, 337, 337, 337, 337, 488, 369, 113,
       337, 246, 417, 180, 246, 246, 246, 369, 246, 246, 180,  19,  19,
        19,  19,  19, 369, 369, 246, 246,   6,   6, 369, 369, 369, 246,
       551, 417, 373, 373, 523, 523, 373, 369, 373, 337, 337, 180, 337,
       337, 374, 337, 705, 337, 337, 705, 180, 180, 417, 417])

In [75]:
one_hot = np.zeros(729)

one_hot[19] = 1
one_hot_to_matrix(one_hot)

array([[0, 0, 2, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [62]:
for r, p in zip(np.argmax(res, axis=1), puz):
    one_hot = np.zeros(729)
    
    one_hot[r] = 1
    ans = one_hot_to_matrix(one_hot)
    
    if (p == 0).sum() / 81 < 10:
        print(ans)
        print 
        print(p)
        print
        print

[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [4 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]

[[0 0 0 0 0 0 2 0 0]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0]
 [0 0 0 6 0 0 0 2 5]
 [0 0 0 0 7 2 0 0 0]
 [2 0 0 0 0 0 0 0 0]
 [0 0 4 0 0 0 0 0 3]
 [0 0 0 0 0 0 0 4 2]]


[[0 0 2 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]

[[0 0 0 0 0 0 2 0 0]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0]
 [0 0 0 6 0 0 0 2 5]
 [0 4 0 0 7 2 0 0 0]
 [2 0 0 0 0 0 0 0 0]
 [0 0 4 0 0 0 0 0 3]
 [0 0 0 0 0 0 0 4 2]]


[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [4 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]

[[0 0 0 0 0 0 2 0 0]
 [0 0 0 0 0 0 0 0 1]
 

In [13]:
non_zero = []
is_empty = []
for r, p in zip(np.argmax(res, axis=1), puz):
    one_hot = np.zeros(729)
    
    one_hot[r] = 1
    ans = one_hot_to_matrix(one_hot)
    
    non_zero.append((p == 0).sum())
    is_empty.append(((p == 0) & (ans != 0) ).sum())

    

In [4]:
input_size = 729
output_size = 729

In [7]:
import tensorflow as tf

tf.reset_default_graph()

x  = tf.placeholder(tf.float32, shape=[None, input_size])
y_ = tf.placeholder(tf.float32, shape=[None, output_size])

w1 = tf.Variable("w1", shape=[input_size, 100])
b1 = tf.Variable(tf.truncated_normal([100], dtype=tf.float32))

w3 = tf.Variable(tf.truncated_normal([100, output_size], dtype=tf.float32))
b3 = tf.Variable(tf.truncated_normal([output_size], dtype=tf.float32))

h1 = tf.nn.sigmoid(tf.matmul(x, w1) + b1)
#h2 = tf.nn.sigmoid(tf.matmul(h1, w2) + b2)
y = tf.matmul(h1, w3) + b3

cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_, logits=y)
train_step = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cross_entropy)

session_conf = tf.ConfigProto(
    device_count={'CPU' : 1, 'GPU' : 0},
    allow_soft_placement=True,
    log_device_placement=True
)

sess = tf.InteractiveSession(config=session_conf)
tf.global_variables_initializer().run()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [8]:
for i in range(10000):
    x_train, y_train = generate_batch(100)
    train_step.run(feed_dict={x: x_train, y_: y_train})

RuntimeError: maximum recursion depth exceeded in cmp

In [9]:
i

0

In [ ]:
x_test, y_test = generate_batch(100)    
res = y.eval(feed_dict={x: x_train})

In [154]:
res.shape

(4, 729)

In [156]:
for am in res.argmax(axis=1):
    r = 

array([136,  19,  72, 120])

In [108]:

tn, fp, fn, tp = confusion_matrix(y_test, res).ravel()

In [109]:
(tn, fp, fn, tp)

(1532, 2, 91, 375)

In [115]:
reload(sudoku)
puzzles = []

for _ in range(10):
    result_steps = sudoku.solve(sudoku.create_matrix())
    puzzles.append(result_steps)

In [121]:
res = puzzles[0][1]

In [122]:
res[1] - res[0]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 9, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
tf.reset_default_graph()

x  = tf.placeholder(tf.float32, shape=[None, input_size])
y_ = tf.placeholder(tf.float32, shape=[None, output_size])


input_weights = []

# the weights for 1 hot encoded input values can be shared as 
# the logic for box, row, column should all be the same
with tf.variable_scope("w") as scope:
    for i in range(9*3):
        
        w = tf.get_variable("weights", [9*9, h1_size],
                initializer=tf.truncated_normal_initializer())
        if i == 0:
            scope.reuse_variables() 
        input_weights.append(w)
        
# concat all the shared weights
w1 = tf.concat(values=input_weights, axis=0)

# normal NN after this point
# perhaps ill add another hidden layer at some point
b1 = tf.Variable(tf.truncated_normal([h1_size], dtype=tf.float32))
h1 = tf.nn.sigmoid(tf.matmul(x, w1) + b1)


w2 = tf.Variable(tf.truncated_normal([h1_size, h2_size], dtype=tf.float32))
b2 = tf.Variable(tf.truncated_normal([h2_size], dtype=tf.float32))
h2 = tf.nn.sigmoid(tf.matmul(h1, w2) + b2)

w3 = tf.Variable(tf.truncated_normal([h2_size, output_size], dtype=tf.float32))
b3 = tf.Variable(tf.truncated_normal([output_size], dtype=tf.float32))



y = tf.matmul(h2, w3) + b3

# this allows us to guess for multiple cells at once
# softmax would allow us only to pick one
# the training data is set up to calculate all possible moves at each step
# so this is important


cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_, logits=y)

#loss = tf.reduce_mean(cross_entropy)

# arbitrary choice, need to learn more about this
train_step = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cross_entropy)

# gpu was OOMing
session_conf = tf.ConfigProto(
    device_count={'CPU' : 1, 'GPU' : 0},
    allow_soft_placement=True,
    log_device_placement=True
)

init_op = tf.global_variables_initializer()